# 3. VGG-19

#### Garrett McCue

The first model architecture we will explore for classifying the images is the [VGG19](https://arxiv.org/abs/1409.1556) architecture. The model architecture and pretrained weights from the ImageNet dataset can easily be downloaded and compiled from [Tensorflow and Keras](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19) for transfer learning applications.

The process of transfer learning for ConvNets includes:

1. Getting only the convolutional layers and associated pretrained weights of the model.
2. Freeze the weights of the pretrained convolutional layers
3. Attach Dense connected layers to the top
4. Train Dense top layers on the desired task. 

In [1]:
import os
# set correct local working directory and useful paths
ROOT = '/Users/garrettmccue/projects/cnn-alzheimers'
data_path = f'{ROOT}/data'
weights_path = f'{ROOT}/models/raw'

os.chdir(ROOT)
os.getcwd()


'/Users/garrettmccue/projects/cnn-alzheimers'

Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/tensorflow/keras-applications/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /Users/garrettmccue/projects/cnn-alzheimers/models/raw/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5

--2022-07-18 17:29:47--  https://storage.googleapis.com/tensorflow/keras-applications/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.176, 172.217.2.48, 142.250.191.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80134624 (76M) [application/octet-stream]
Saving to: ‘/Users/garrettmccue/projects/cnn-alzheimers/models/raw/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5’

/Users/garrettmccue 100%[===================>]  76.42M  43.8MB/s    in 1.7s    

2022-07-18 17:29:49 (43.8 MB/s) - ‘/Users/garrettmccue/projects/cnn-alzheimers/models/raw/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [80134624/80134624]



In [3]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

# set the downloaded weights file to a variable
local_weights_path = f'{weights_path}/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize base model from tensorflow.keras
# Set the input shape and remove dense layers (top), to match our weights file
# our image shapes are (256x256)
pre_trained_model = VGG19(input_shape = (256, 256, 3),
                                include_top= False,
                                weights = None)

# load pretrained weights
pre_trained_model.load_weights(local_weights_path)

# freeze weights of each layer
for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

2022-07-18 17:31:47.303327: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

Since the head has been removed and this network isnt as deep as the inception-v3 model we will take the output from the last layer and feed it into an attached dense network. Since we are going to be adding dense networks to more models we can create a utilities function to handle this and use it to add the dense layers to the vgg19 model.

In [ ]:
# initiate the output of the `block5_pool` layer so it can be fed into the dense network.
LAST_LAYER = 'block5_pool'
print(f'Setting last layer as: {LAST_LAYER}')
last_layer = pre_trained_model.get_layer(LAST_LAYER)
print(f'Last layer output shape: {last_layer.output_shape}')
last_output = last_layer.output


def add_Dense_layers(model, last_output, num_nodes, class_num, dropout):
    #TODO
    print('Building Dense layers...')
    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # add a fully connected layer
    x = layers.Dense(num_nodes, activation='relu')(x)
    # add the dropout rate of 0.2
    x = layers.Dropout(dropout)(x)
    # add a final layer with softmax for classification
    x = layers.Dense(class_num, activation='softmax')(x)

    # append the created dense network to the pre_trained_model
    dense_model = Model(model.input, x)


    return dense_model

The code to construct the model can be written into the src.models.vgg19.py module to be used for training the models. The dense network method can be moved into the src.models.model_utils.py module to be used when creating each of the model architectures. This method will be called within each of the model modules.

In [ ]:
#TODO import vgg19 module and compile a model showing the summary